<a href="https://colab.research.google.com/github/uniquing/To-become-a-junior-data-analysis/blob/main/Junior_analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### XML擷取

In [ ]:
import requests
# 利用7-11電子地圖查看台北門市
# 進行 POST 請求時要攜帶資料
form_data = {
    "commandid": "GetTown",
    "cityid": "01"
}
request_url = "https://emap.pcsc.com.tw/EMapSDK.aspx"
response = requests.post(request_url, data=form_data)

In [ ]:
response_content=response.content
print(response_content)

b'<?xml version="1.0" encoding="utf-8"?><iMapSDKOutput><MessageID>00000</MessageID><CommandID>GetTown</CommandID><Status>\xe9\x80\xa3\xe7\xb7\x9a\xe6\x88\x90\xe5\x8a\x9f</Status><TimeStamp>2020/5/31 \xe4\xb8\x8b\xe5\x8d\x88 02:28:36</TimeStamp><GeoPosition><TownID>01</TownID><TownName>\xe6\x9d\xbe\xe5\xb1\xb1\xe5\x8d\x80</TownName><X>121577218</X><Y>25049837</Y></GeoPosition><GeoPosition><TownID>02</TownID><TownName>\xe4\xbf\xa1\xe7\xbe\xa9\xe5\x8d\x80</TownName><X>121567161</X><Y>25033147</Y></GeoPosition><GeoPosition><TownID>03</TownID><TownName>\xe5\xa4\xa7\xe5\xae\x89\xe5\x8d\x80</TownName><X>121534593</X><Y>25026482</Y></GeoPosition><GeoPosition><TownID>04</TownID><TownName>\xe4\xb8\xad\xe5\xb1\xb1\xe5\x8d\x80</TownName><X>121533655</X><Y>25064427</Y></GeoPosition><GeoPosition><TownID>05</TownID><TownName>\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80</TownName><X>121518245</X><Y>25032251</Y></GeoPosition><GeoPosition><TownID>06</TownID><TownName>\xe5\xa4\xa7\xe5\x90\x8c\xe5\x8d\x80</TownNa

In [ ]:
print(type(response_content))

<class 'bytes'>


In [ ]:
from lxml import etree
from io import BytesIO

file = BytesIO(response_content)
tree = etree.parse(file)
Towns_names = [e.text for e in tree.xpath("//TownName")]
print(Towns_names)

['松山區', '信義區', '大安區', '中山區', '中正區', '大同區', '萬華區', '文山區', '南港區', '內湖區', '士林區', '北投區']


In [ ]:
import requests
from lxml import etree
from io import BytesIO
#查看台北松山區
#進行 POST 請求時要攜帶資料
form_data = {
    "commandid": "SearchRoad",
    "city": "台北市",
    "town": "松山區"
}
request_url = "https://emap.pcsc.com.tw/EMapSDK.aspx"
response = requests.post(request_url, data=form_data)

file = BytesIO(response.content)
tree = etree.parse(file)
Road_names = [e.text for e in tree.xpath("//rd_name_1")]
print(Road_names)

['八德路', '八德路', '八德路', '三民路', '北寧路', '市民大道', '民生東路', '民生東路', '民生東路', '民權東路', '光復北路', '延吉街', '延壽街', '東興路', '南京東路', '南京東路', '南京東路', '健康路', '復興北路', '復興南路', '敦化北路', '敦化南路', '塔悠路', '新東街', '寧安街', '慶城街', '撫遠街', '寶清街']


In [ ]:
import requests
from lxml import etree
from io import BytesIO
#查看八德路段有哪些門市
#進行 POST 請求時要攜帶資料
form_data = {
    "commandid": "SearchStore",
    "city": "台北市",
    "town": "松山區",
    "roadname":"八德路二段"
}
request_url = "https://emap.pcsc.com.tw/EMapSDK.aspx"
response = requests.post(request_url, data=form_data)

file = BytesIO(response.content)
tree = etree.parse(file)
Store_names = [e.text for e in tree.xpath("//POIName")]
print(Store_names)

['庫德', '清愿']


In [ ]:
from lxml import etree
from io import BytesIO

file = BytesIO(response_content)
tree = etree.parse(file)
town_names = [t.text for t in tree.xpath("//TownName")] # XPath 亦可以指定 /iMapSDKOutput/GeoPosition/TownName
print(town_names)

['松山區', '信義區', '大安區', '中山區', '中正區', '大同區', '萬華區', '文山區', '南港區', '內湖區', '士林區', '北投區']


In [ ]:
#寫成完整Code

import time
import random

tp_711_stores = {}
for town in town_names:
    form_data = {
        "commandid": "SearchStore",
        "city": "台北市",
        "town": town
    }
    r = requests.post("https://emap.pcsc.com.tw/EMapSDK.aspx", data=form_data)
    f = BytesIO(r.content)
    tree = etree.parse(f)
    poi_ids = [t.text.strip() for t in tree.xpath("//POIID")]
    poi_names = [t.text for t in tree.xpath("//POIName")]
    lons = [float(t.text)/1000000 for t in tree.xpath("//X")]
    lats = [float(t.text)/1000000 for t in tree.xpath("//Y")]
    adds = [t.text for t in tree.xpath("//Address")]
    tp_711_stores[town] = []
    for poi_id, poi_name, lon, lat, add in zip(poi_ids, poi_names, lons, lats, adds):
        store_info = {
            "POIID": poi_id,
            "POIName": poi_name,
            "Longitude": lon,
            "Latitude": lat,
            "Address": add
        }
        tp_711_stores[town].append(store_info)
    time.sleep(random.randint(1, 6))
    print("Scraping {}".format(town))

Scraping 松山區
Scraping 信義區
Scraping 大安區
Scraping 中山區
Scraping 中正區
Scraping 大同區
Scraping 萬華區
Scraping 文山區
Scraping 南港區
Scraping 內湖區
Scraping 士林區
Scraping 北投區


In [ ]:
tp_711_stores["松山區"][2]

{'Address': '台北市松山區八德路三段27號',
 'Latitude': 25.048396,
 'Longitude': 121.552737,
 'POIID': '960661',
 'POIName': '中崙'}

### 擷取IMDB網頁資料

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_movie_data(movie_url):
    movie_title_cs="h1"
    movie_poster_cs=".poster img"
    movie_rating_cs="strong span"
    movie_release_cs=".subtext a"
    movie_cast_cs=".primary_photo+ td a"
    
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text)
    
    movie_title= soup.select(movie_title_cs)[0].text.replace("\xa0"," ").strip()
    
    movie_poster=soup.select(movie_poster_cs)[0].get("src")

    movie_rating=float(soup.select(movie_rating_cs)[0].text)

    genre=[soup.text for soup in soup.select(movie_release_cs)][0:3]
    
    movie_release=soup.select(movie_release_cs)[3].text.rstrip()

    movie_cast=[e.text.strip() for e in soup.select(movie_cast_cs)]

    movie_data={
        "movieTitle": movie_title,
        "moviePoster": movie_poster,
        "movieRating": movie_rating,
        "movieGenre": genre,
        "movieReleaseDate": movie_release,
        "movie_cast": movie_cast
    }
    return movie_data

### 利用關鍵字取代網頁搜尋

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_movie_data(name):
    movie_title_cs="h1"
    movie_poster_cs=".poster img"
    movie_rating_cs="strong span"
    movie_release_cs=".subtext a"
    movie_cast_cs=".primary_photo+ td a"
    result_text_cs=".result_text > a"

    query_string_parameters = {
    'q': name,
    'ref_': 'nv_sr_sm'
    }

    request_url= "https://www.imdb.com/find"
    response=requests.get(request_url, params=query_string_parameters) 
    soup= BeautifulSoup(response.text)

    result_text_href = [e.get("href") for e in soup.select(result_text_cs)]
    movie_href=result_text_href[0]
    movie_url="http://www.imdb.com{}".format(movie_href)

    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text)
    
    movie_title= soup.select(movie_title_cs)[0].text.replace("\xa0"," ").strip()
    
    movie_poster=soup.select(movie_poster_cs)[0].get("src")

    movie_rating=float(soup.select(movie_rating_cs)[0].text)

    genre=[soup.text for soup in soup.select(movie_release_cs)][0:3]
    
    movie_release=soup.select(movie_release_cs)[3].text.rstrip()

    movie_cast=[e.text.strip() for e in soup.select(movie_cast_cs)]

    movie_data={
        "movieTitle": movie_title,
        "moviePoster": movie_poster,
        "movieRating": movie_rating,
        "movieGenre": genre,
        "movieReleaseDate": movie_release,
        "movie_cast": movie_cast
    }
    return movie_data



In [ ]:
get_movie_data("https://www.imdb.com/title/tt0451279/")

{'movieGenre': ['Action', 'Adventure', 'Fantasy'],
 'moviePoster': 'https://m.media-amazon.com/images/M/MV5BNDFmZjgyMTEtYTk5MC00NmY0LWJhZjktOWY2MzI5YjkzODNlXkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_UX182_CR0,0,182,268_AL_.jpg',
 'movieRating': 7.4,
 'movieReleaseDate': '2 June 2017 (USA)',
 'movieTitle': 'Wonder Woman (2017)',
 'movie_cast': []}

In [ ]:
get_movie_data("wonder woman")

{'movieGenre': ['Action', 'Adventure', 'Fantasy'],
 'moviePoster': 'https://m.media-amazon.com/images/M/MV5BNDFmZjgyMTEtYTk5MC00NmY0LWJhZjktOWY2MzI5YjkzODNlXkEyXkFqcGdeQXVyMDA4NzMyOA@@._V1_UX182_CR0,0,182,268_AL_.jpg',
 'movieRating': 7.4,
 'movieReleaseDate': '2 June 2017 (USA)',
 'movieTitle': 'Wonder Woman (2017)',
 'movie_cast': []}

In [ ]:
get_movie_data("Avenger: Endgame")

{'movieGenre': ['Action', 'Adventure', 'Drama'],
 'moviePoster': 'https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_UX182_CR0,0,182,268_AL_.jpg',
 'movieRating': 8.4,
 'movieReleaseDate': '26 April 2019 (USA)',
 'movieTitle': 'Avengers: Endgame (2019)',
 'movie_cast': []}

### 擷取華航機上所有電影清單

In [ ]:
import requests
from bs4 import BeautifulSoup
request_url="http://www.fantasy-sky.com/ContentList.aspx"

airline_movie_cs=".movies-name"

airline_cookies={
    "COOKIE_LANGUAGE":"en"
}

ca_movies_name=[]
for i in range(1,5):
  query_string_parameters={
      "section": "002",
      "category": "0020{}".format(i)
      }

  response = requests.get(request_url, params=query_string_parameters, cookies=airline_cookies)
  soup=BeautifulSoup(response.text)
  movies_name=[e.text for e in soup.select(airline_movie_cs)]
  ca_movies_name+=movies_name

print(ca_movies_name)
print(len(ca_movies_name))



['1917', 'Just Mercy', 'Wendy', 'Bad Boys for Life', 'Disney and Pixar’s Onward', 'The Call of the Wild', 'Star Wars: The Rise of Skywalker', 'Sonic the Hedgehog', 'Somewhere Winter', 'Detention', 'Little Women', 'Dark Waters', 'Spies in Disguise', 'The Good Liar', 'The Informer', 'Underwater', 'Talking the Pictures', 'Kim Ji-young: Born 1982', 'Miyamoto', 'One Piece Stampede', 'Special Actors', 'We Are Little Zombies', 'Better Days', 'Guilt By Design', 'Synapses', 'At the End of the Matinee', 'Parasite', 'Oh Mercy!', 'Cheer Up, Mr. Lee', 'Uri: The Surgical Strike', 'Papicha', 'The Specials', 'The Book of Life', 'Big Hero 6', 'Night at the Museum: Secret…', 'Frozen', 'Disney and Pixar’s Finding Dory', 'Shark Tale', 'DISNEY AND PIXAR’S Inside Out', 'Disney and Pixar’s The Good Dinosaur', "Tim Burton's Corpse Bride", 'Smallfoot', 'Disney and Pixar’s Incredibles 2', 'Ferdinand', 'A Sun', 'A Witness out of the Blue', 'Jade Dynasty', 'Fagara', 'Integrity', 'The Last Thieves', "Aesop's Game"

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_movie_data(name):
    movie_title_cs="h1"
    movie_poster_cs=".poster img"
    movie_rating_cs="strong span"
    movie_release_cs=".subtext a"
    movie_cast_cs=".primary_photo+ td a"
    result_text_cs=".result_text > a"

    query_string_parameters = {
    'q': name,
    's': 'tt',
    'ref_': 'fn_tt'   
    }

    request_url= "https://www.imdb.com/find"
    response=requests.get(request_url, params=query_string_parameters) 
    soup= BeautifulSoup(response.text)

    result_text_href = [e.get("href") for e in soup.select(result_text_cs)]
    movie_href=result_text_href[0]
    movie_url="http://www.imdb.com{}".format(movie_href)

    response = requests.get(movie_url)
    soup = BeautifulSoup(response.text)
    
    movie_title= soup.select(movie_title_cs)[0].text.replace("\xa0"," ").strip()
    
    movie_poster=soup.select(movie_poster_cs)[0].get("src")

    movie_rating_cs_element_tag=soup.select(movie_rating_cs)[0]
    movie_rating=float(movie_rating_cs_element_tag.text)
   
    genre_all=[]
    for soup in soup.select(movie_release_cs):
      genre_all.append(soup.text)
    length=len(genre_all)
    genre=genre_all[0:length-1]   

    movie_release=genre_all[-1].rstrip()

    movie_cast=[e.text.strip() for e in soup.select(movie_cast_cs)]

    movie_data={
        "movieTitle": movie_title,
        "moviePoster": movie_poster,
        "movieRating": movie_rating,
        "movieGenre": genre,
        "movieReleaseDate": movie_release,
        "movie_cast": movie_cast
    }
    return movie_data

In [ ]:
get_movie_data("Bad Boys For Life (2020)")

{'movieGenre': ['Action', 'Comedy', 'Crime'],
 'moviePoster': 'https://m.media-amazon.com/images/M/MV5BMWU0MGYwZWQtMzcwYS00NWVhLTlkZTAtYWVjOTYwZTBhZTBiXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UX182_CR0,0,182,268_AL_.jpg',
 'movieRating': 6.7,
 'movieReleaseDate': '17 January 2020 (USA)',
 'movieTitle': 'Bad Boys For Life (2020)',
 'movie_cast': []}

In [ ]:
imdb_data=[]
for movie in ca_movies_name:
    try:
      imdb_movie_data=get_movie_data(movie)
      imdb_data.append(imdb_movie_data)
      imdb_movie_title=imdb_movie_data["movieTitle"]
      imdb_movie_rating=imdb_movie_data["movieRating"]
      print(imdb_movie_title,imdb_movie_rating)
    except:
      print(" {}發生錯誤...".format(movie))

1917 (2019) 8.3
Just Mercy (2019) 7.5
Wendy (2020) 5.6
Bad Boys For Life (2020) 6.7
 Disney and Pixar’s Onward發生錯誤...
The Call of the Wild (2020) 6.8
Star Wars: Episode IX - The Rise of Skywalker (2019) 6.7
Sonic the Hedgehog (2020) 6.6
Da yue zai dong ji (2019) 5.7
Detention (2011) 5.7
Little Women (2019) 7.9
Dark Waters (2019) 7.6
Spies in Disguise (2019) 6.8
The Good Liar (2019) 6.6
The Informer (2019) 6.5
Underwater (2020) 5.8
Katsuben! (2019) 6.4
Kim Ji-young: Born 1982 (2019) 7.4
 Miyamoto發生錯誤...
One Piece: Stampede (2019) 7.7
Special Actors (2019) 5.9
We Are Little Zombies (2019) 6.9
Better Days (2019) 7.5
Guilt by Design (2019) 5.9
Synapses (2019) 6.6
 At the End of the Matinee發生錯誤...
Parasite (2019) 8.6
Oh Mercy! (2019) 6.2
Himeul naeyo, Miseuteo Lee (2019) 6.8
Uri: The Surgical Strike (2019) 8.3
Papicha (2019) 7.0
The Specials (2019) 7.5
The Book of Life (2014) 7.3
Big Hero 6 (2014) 7.8
Night at the Museum: Secret of the Tomb (2014) 6.2
Frozen (2013) 7.4
 Disney and Pixar’s F

In [ ]:
movie_titles,movie_ratings=[], []
for data in imdb_data:
  movie_titles.append(data["movieTitle"])
  movie_ratings.append(data["movieRating"])
print(len(movie_titles))
print(len(movie_ratings))

126
126


In [ ]:
max_rating=max(movie_ratings)
print(max_rating)
The_index_best=[]
for index, rating in enumerate(movie_ratings):
  if rating==max_rating:
    print("位於{}的評分最高".format(index))
    The_index_best.append(index)
print(movie_titles[The_index_best[0]])    

9.3
位於123的評分最高
The Shawshank Redemption (1994)


In [ ]:
import pandas as pd

movie_df=pd.DataFrame()
movie_df['movie_title']=movie_titles
movie_df["movie_rating"]=movie_ratings
movie_df.sort_values("movie_rating",ascending=False)

,movie_title,movie_rating
123,The Shawshank Redemption (1994),9.3
90,Inception (2010),8.8
23,Parasite (2019),8.6
82,Joker (2019),8.5
83,Avengers: Infinity War (2018),8.5
...,...,...
2,Wendy (2020),5.7
119,R.I.P.D. (2013),5.6
53,All About Yves (2019),5.5
41,Integrity (2019),5.5


In [ ]:
def get_movie_data(search_movie_title):
    from selenium import webdriver
    #Turn on the firefox driver from desktop
    driver_path = "C:/Users/uniquing/Desktop/geckodriver.exe"
    imdb_home = "https://www.imdb.com/"
    driver = webdriver.Firefox(executable_path=driver_path) # Use Firefox
    driver.get(imdb_home)
    #find out the xpath from all section
    movie_title_xpth="//h1"
    movie_poster_xpth="//div[@class='poster']/a/img/@src"
    movie_rating_xpth="//div[@class='ratingValue']/strong/span"
    movie_release_xpth="//div[@class='subtext']/a"
    movie_cast_xpth="//td[2]/a"
    imdb_search_input_xpth="//input[@id='suggestion-search']"
    imdb_search_button_xpth="//button[@id='suggestion-search-button']"
    imdb_catagory_button_xpth="//ul[@class='findTitleSubfilterList']/li[1]/a"
    imdb_results_list_xpth="//td[@class='result_text']/a"

    #send request to Firefox
    elem=driver.find_element_by_xpath(imdb_search_input_xpth)
    #enter the search keys
    elem.send_keys(search_movie_title)
    #find out the search button
    elem=driver.find_element_by_xpath(imdb_search_button_xpth)
    elem.click()
    #filter the results through catagory
    elem=driver.find_element_by_xpath(imdb_catagory_button_xpth)
    elem.click()
    #find the results list
    elems=driver.find_elements_by_xpath(imdb_results_list_xpth)
    first_elems=elems[0]
    first_elems.click()

In [ ]:
#movie_title
elem=driver.find_element_by_xpath("movie_title_xpth")
movie_title=elem.text
#movie_poster
elem=driver.find_element_by_xpath("//div[@class='poster']/a/img")
movie_poster=elem.get_attribute("src")
#movie rating
elem=driver.find_element_by_xpath("//div[@class='ratingValue']/strong/span")
movie_rating=float(elem.text)

In [ ]:
elems=driver.find_elements_by_xpath("//div[@class='subtext']/a")
movie_release=[e.text for e in elems]

elems=driver.find_elements_by_xpath("//td[2]/a")
movie_cast=[e.text for e in elems]
movie_data={
    "movieTitle":movie_title,
    "moviePoster":movie_poster,
    "movieRating":movie_rating,
    "movieRelease":movie_release,
    "movieCast":movie_cast
}
return movie_data

#  擷取Yahoo Stock資料


In [ ]:
!pip install -U beautifulsoup4

Requirement already up-to-date: beautifulsoup4 in /usr/local/lib/python3.6/dist-packages (4.9.1)


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_price_rank():
  query_string_parameters={
        "t": "pri",
        "e": "tse",
        "n": "100"
    }
  url = "https://tw.stock.yahoo.com/d/i/rank.php"
  response = requests.get(url,params=query_string_parameters)
  soup = BeautifulSoup(response.text)

  ticker_cs = ".name a"
  price_cs=".name+ td"

  ticker_names=[e.text for e in soup.select(ticker_cs)]
  tickers=[i.split(" ")[0] for i in ticker_names]
  stock_names=[i.split(" ")[1] for i in ticker_names]
  prices = [e.text for e in soup.select(price_cs) ]
  
  return tickers, stock_names, prices

In [ ]:
tickers, stock_names, prices = get_price_rank()
print(tickers)
print(stock_names)
print(prices)

['3008', '6415', '5269', '6409', '6669', '2207', '8454', '3406', '1590', '2454', '2327', '3533', '3661', '1476', '2059', '3563', '2330', '2049', '2912', '2395', '6531', '2227', '2379', '6504', '3653', '8341', '8464', '6230', '9921', '3443', '2357', '2474', '2404', '2345', '1256', '6414', '3665', '3034', '4137', '1325', '1707', '8462', '2492', '9910', '9914', '6670', '3413', '4551', '4968', '6491', '8070', '3130', '8422', '2231', '1477', '2308', '2707', '8016', '8081', '2383', '3023', '6452', '6271', '1537', '3532', '4439', '3515', '6213', '6706', '2360', '2439', '4119', '6715', '3376', '6416', '4766', '4438', '4958', '6666', '1558', '6598', '2939', '8480', '4935', '2458', '4977', '5871', '6533', '3044', '4536', '1760', '1232', '3026', '4763', '6456', '4943', '2723', '6269', '6278', '6464']
['大立光', '矽力-KY', '祥碩', '旭隼', '緯穎', '和泰車', '富邦媒', '玉晶光', '亞德客-KY', '聯發科', '國巨', '嘉澤', '世芯-KY', '儒鴻', '川湖', '牧德', '台積電', '上銀', '統一超', '研華', '愛普', '裕日車', '瑞昱', '南六', '健策', '日友', '億豐', '超眾', '巨大', '創意', 

In [ ]:
from statistics import median
ky_prices = [price for stock_name, price in zip(stock_names, prices) if "KY" in stock_name]
print(median(ky_prices))
print(len(ky_prices))

#  NBA球員資料分析

In [ ]:
def recode_pos(x):
    if x[0] == 'G':
        return 'G'
    elif x[0] == 'F':
        return 'F'
    elif x[0] == 'C':
        return 'C'

In [ ]:
player_profile["pos_recoded"]=player_profile["pos"].apply(recode_pos)
player_profile["pos_recoded"].value_counts()

G    239
F    221
C     64
Name: pos_recoded, dtype: int64

In [ ]:
player_profile.head()

,collegeName,country,dateOfBirthUTC,firstName,heightFeet,heightInches,heightMeters,isActive,jersey,lastAffiliation,lastName,nbaDebutYear,personId,pos,teamId,temporaryDisplayName,weightKilograms,weightPounds,yearsPro,BMI,height_catagory,pos_recoded
0,Pittsburgh,New Zealand,1993-07-20,Steven,7,0,2.13,1,12.0,Pittsburgh/New Zealand,Adams,2013.0,203500,C,1610612760,"Adams, Steven",120.2,265.0,6.0,26.493861,>2m,C
1,Kentucky,USA,1997-07-18,Bam,6,10,2.08,1,13.0,Kentucky/USA,Adebayo,2017.0,1628389,C-F,1610612748,"Adebayo, Bam",115.7,255.0,2.0,26.742788,>2m,C
2,University of Louisville,South Sudan,1997-02-01,Deng,6,7,2.01,1,NaN,South Sudan/South Sudan,Adel,2018.0,1629061,F,1610612751,"Adel, Deng",90.7,200.0,1.0,22.449939,>2m,F
3,Texas,USA,1985-07-19,LaMarcus,6,11,2.11,1,12.0,Texas/USA,Aldridge,2006.0,200746,C-F,1610612759,"Aldridge, LaMarcus",117.9,260.0,13.0,26.481885,>2m,C
4,Tennessee,Canada,1996-10-21,Kyle,6,11,2.11,1,NaN,Tennessee/Canada,Alexander,NaN,1629734,F,1610612748,"Alexander, Kyle",99.8,220.0,0.0,22.416388,>2m,F


In [ ]:
import pandas as pd

numbers = [9, 23, 33, 91, 13]
players = ["Ron Harper", "Michael Jordan", "Scottie Pippen", "Dennis Rodman", "Luc Longley"]
df = pd.DataFrame()
df["number"] = numbers
df["player"] = players
df=df.set_index("player")
df

,number
player,
Ron Harper,9
Michael Jordan,23
Scottie Pippen,33
Dennis Rodman,91
Luc Longley,13


In [ ]:

print("whose number is smallest:",df["number"].idxmin())

whose number is smallest: Ron Harper


In [ ]:
import numpy as ny
import pandas as pd

In [ ]:
player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
player_profile["BMI"]=player_profile["weightKilograms"]/player_profile["heightMeters"]**2
player_profile[["temporaryDisplayName", "BMI"]].head()

,temporaryDisplayName,BMI
0,"Adams, Steven",26.493861
1,"Adebayo, Bam",26.742788
2,"Adel, Deng",22.449939
3,"Aldridge, LaMarcus",26.481885
4,"Alexander, Kyle",22.416388


In [ ]:
player_profile["country"].unique()

array(['New Zealand', 'USA', 'South Sudan', 'Canada', 'Greece',
       'United Kingdom', 'Bahamas', 'France', 'Australia', 'Italy',
       'Croatia', 'Latvia', 'Georgia', 'Democratic Republic of the Congo',
       'Serbia', 'Sudan', 'Germany', 'Lithuania', 'Brazil', 'Slovenia',
       'Switzerland', nan, 'Mali', 'Senegal', 'Guinea', 'Cameroon',
       'Angola', ' ', 'Spain', 'Yugoslavia', 'Japan',
       'Dominican Republic', 'Turkey', 'Haiti', 'Ukraine', 'Finland',
       'Bosnia and Herzegovina', 'Egypt', 'Nigeria', 'Austria',
       'Czech Republic', 'Gabon', 'Montenegro'], dtype=object)

In [ ]:
player_profile["country"].nunique()

42

In [ ]:
player_profile["country"].value_counts()

In [ ]:
group_object = player_profile.groupby("pos") #按照postion分組

# 對投票所票數資料整理


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
from urllib.parse import quote
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls"
file_name_url = quote(file_name)
print(file_name_url)

%E7%B8%BD%E7%B5%B1-A05-4-%E5%80%99%E9%81%B8%E4%BA%BA%E5%BE%97%E7%A5%A8%E6%95%B8%E4%B8%80%E8%A6%BD%E8%A1%A8-%E5%90%84%E6%8A%95%E9%96%8B%E7%A5%A8%E6%89%80%28%E6%96%B0%E5%8C%97%E5%B8%82%29.xls


In [ ]:
from urllib.parse import quote
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_name_url = quote(file_name)
print(file_name_url)

%E7%B8%BD%E7%B5%B1-A05-4-%E5%80%99%E9%81%B8%E4%BA%BA%E5%BE%97%E7%A5%A8%E6%95%B8%E4%B8%80%E8%A6%BD%E8%A1%A8-%E5%90%84%E6%8A%95%E9%96%8B%E7%A5%A8%E6%89%80%28%E8%87%BA%E5%8C%97%E5%B8%82%29.xls


In [ ]:
import pandas as pd
file_path = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_name_url)
xls_df = pd.read_excel(file_path, skiprows=[0, 1, 3, 4])
xls_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,總 計,NaN,NaN,"70,769","685,830","875,854","1,632,453","21,381","1,653,834",143,"1,653,977","513,287","2,167,264",76.3098
1,松山區,NaN,NaN,"5,436","55,918","64,207","125,561","1,762","127,323",2,"127,325","37,329","164,654",77.3276
2,NaN,莊敬里,573.0,36,391,429,856,14,870,0,870,230,"1,100",79.0909
3,NaN,莊敬里,574.0,46,382,438,866,12,878,0,878,259,"1,137",77.2208
4,NaN,莊敬里,575.0,48,393,389,830,22,852,0,852,262,"1,114",76.4811


In [ ]:
columns_names=list(xls_df.columns)
n_candidates=len(columns_names)-11
candidates_n_names=columns_names[3:(3+n_candidates)]
print(candidates_n_names)

['(1)\n宋楚瑜\n余湘', '(2)\n韓國瑜\n張善政', '(3)\n蔡英文\n賴清德']


In [ ]:
from string import ascii_uppercase
column_names = ["district", "village", "office"] + candidates_n_names + list(ascii_uppercase[:8])
xls_df.columns = column_names
xls_df.head()

In [ ]:
district_filled=xls_df["district"].fillna(method="ffill")
district_filled

In [ ]:
xls_df=xls_df.drop("district",axis=1 )
xls_df.insert(0,"district",district_filled)
xls_df

In [ ]:
xls_df["district"]=xls_df["district"].str.replace("\u3000","").str.strip()
print(xls_df["district"].unique())

['總計' '松山區' '信義區' '大安區' '中山區' '中正區' '大同區' '萬華區' '文山區' '南港區' '內湖區' '士林區'
 '北投區']


In [ ]:
xls_df=xls_df.dropna().reset_index(drop=True)
xls_df.head()

,district,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
0,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,"1,100",79.0909
1,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,"1,137",77.2208
2,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,"1,114",76.4811
3,松山區,莊敬里,576.0,43,389,462,894,14,908,0,908,271,"1,179",77.0144
4,松山區,東榮里,577.0,38,431,545,"1,014",18,"1,032",0,"1,032",272,"1,304",79.1411


In [ ]:
import pandas as pd
from string import ascii_uppercase

def get_tidy_data(file_name):
  
  xls_df = pd.read_excel(file_name, skiprows=[0, 1, 3, 4])
  #找到欄位候選人號碼
  columns_names=list(xls_df.columns)
  n_candidates=len(columns_names)-11
  candidates_n_names=columns_names[3:(3+n_candidates)]
 
  #將欄位命名
  
  column_names = ["district", "village", "office"] + candidates_n_names + list(ascii_uppercase[:8])
  xls_df.columns = column_names
  #將行政區NA值用先前值填補
  district_filled=xls_df["district"].fillna(method="ffill")
  #行政區有NA值,將完整行政區填入
  xls_df=xls_df.drop("district",axis=1 )
  xls_df.insert(0,"district",district_filled)
  #整理行政區文字
  xls_df["district"]=xls_df["district"].str.replace("\u3000","").str.strip()
  #捨棄總計和小計剛好是NA值所在列
  tidy_data=xls_df.dropna().reset_index(drop=True)

  return tidy_data

In [ ]:
from urllib.parse import quote
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls"
file_name_url = quote(file_name)
file_path = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_name_url)
get_tidy_data(file_path).head()

In [ ]:
admin_areas = ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市", "新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣", "屏東縣", "宜蘭縣", "花蓮縣", "臺東縣", "澎湖縣", "基隆市", "新竹市", "嘉義市", "金門縣", "連江縣"]
file_names = ["總統-A05-4-候選人得票數一覽表-各投開票所({}).xls".format(areas) for areas in admin_areas]
file_names

In [ ]:
df_dict={}
for file_name, admin_area in zip(file_names, admin_areas):
  tidy_df = get_tidy_data(file_name)
  df_dict[admin_area] = tidy_df
  print("現在正在處理{}的資料...".format(admin_area))
  print("資料外觀為：", tidy_df.shape)

In [ ]:
df_dict["臺北市"]

In [ ]:
tpe=df_dict["臺北市"]

tpe=tpe[tpe.columns[:6]]#tpe.drop(list(ascii_uppercase[:8]),axis=1)

In [ ]:
candidate_infos = list(tpe.columns[3:])
tpe = tpe.melt(id_vars=['district', 'village', 'office'], value_vars=candidate_infos, var_name="candidate_info", value_name='votes')
tpe.head()

,district,village,office,candidate_info,votes
0,松山區,莊敬里,573.0,(1)\n宋楚瑜\n余湘,36
1,松山區,莊敬里,574.0,(1)\n宋楚瑜\n余湘,46
2,松山區,莊敬里,575.0,(1)\n宋楚瑜\n余湘,48
3,松山區,莊敬里,576.0,(1)\n宋楚瑜\n余湘,43
4,松山區,東榮里,577.0,(1)\n宋楚瑜\n余湘,38


In [ ]:
presidential_votes=pd.DataFrame()
for admin_area in admin_areas:
  county=df_dict[admin_area]
  county=county[county.columns[:6]]
  county = county.melt(id_vars=['district', 'village', 'office'], value_vars=candidate_infos, var_name="candidate_info", value_name='votes')
  county["admin_area"]=admin_area
  presidential_votes=presidential_votes.append(county)

presidential_votes = presidential_votes.reset_index(drop=True)

In [ ]:
presidential_votes.head()

,district,village,office,candidate_info,votes,admin_area
0,松山區,莊敬里,573.0,(1)\n宋楚瑜\n余湘,36,臺北市
1,松山區,莊敬里,574.0,(1)\n宋楚瑜\n余湘,46,臺北市
2,松山區,莊敬里,575.0,(1)\n宋楚瑜\n余湘,48,臺北市
3,松山區,莊敬里,576.0,(1)\n宋楚瑜\n余湘,43,臺北市
4,松山區,東榮里,577.0,(1)\n宋楚瑜\n余湘,38,臺北市


In [ ]:
split_candidate=presidential_votes["candidate_info"].str.split("\n",expand=True)
presidential_votes["number"]=split_candidate[0].str.replace("\(","").str.replace("\)","")
presidential_votes["candidate_info"]=split_candidate[1].str.cat(split_candidate[2],"/")
presidential_votes.head()

,district,village,office,candidate_info,votes,admin_area,number
0,松山區,莊敬里,573.0,宋楚瑜/余湘,36,臺北市,1
1,松山區,莊敬里,574.0,宋楚瑜/余湘,46,臺北市,1
2,松山區,莊敬里,575.0,宋楚瑜/余湘,48,臺北市,1
3,松山區,莊敬里,576.0,宋楚瑜/余湘,43,臺北市,1
4,松山區,東榮里,577.0,宋楚瑜/余湘,38,臺北市,1


In [ ]:
def get_party(number):
  party=["親民黨","中國國民黨","民主進步黨"]
  if number == '1':
    return party[0]
  elif number == '2':
    return party[1]
  elif number == '3':
    return party[2]
  

In [ ]:
presidential_votes['party'] = presidential_votes['number'].apply(get_party)

In [ ]:
presidential_votes.head()

,district,village,office,candidate_info,votes,admin_area,number,party
0,松山區,莊敬里,573.0,宋楚瑜/余湘,36,臺北市,1,親民黨
1,松山區,莊敬里,574.0,宋楚瑜/余湘,46,臺北市,1,親民黨
2,松山區,莊敬里,575.0,宋楚瑜/余湘,48,臺北市,1,親民黨
3,松山區,莊敬里,576.0,宋楚瑜/余湘,43,臺北市,1,親民黨
4,松山區,東榮里,577.0,宋楚瑜/余湘,38,臺北市,1,親民黨


In [ ]:
presidential_votes = presidential_votes[["admin_area","district", "village", "office", "number","party", "candidate_info", "votes"]]


In [ ]:
presidential_votes.head()

,admin_area,district,village,office,number,party,candidate_info,votes
0,臺北市,松山區,莊敬里,573,1,親民黨,宋楚瑜/余湘,36
1,臺北市,松山區,莊敬里,574,1,親民黨,宋楚瑜/余湘,46
2,臺北市,松山區,莊敬里,575,1,親民黨,宋楚瑜/余湘,48
3,臺北市,松山區,莊敬里,576,1,親民黨,宋楚瑜/余湘,43
4,臺北市,松山區,東榮里,577,1,親民黨,宋楚瑜/余湘,38


In [ ]:
presidential_votes.dtypes

In [ ]:
presidential_votes['number'] = presidential_votes['number'].astype(int)
presidential_votes['office'] = presidential_votes['office'].astype(int)
presidential_votes['votes'] = presidential_votes['votes'].astype(str)
presidential_votes['votes'] = presidential_votes['votes'].str.replace(',', '').astype(int)